🕺 This ipynb file is draft for aggregation models for free events about IT and programming: webinars, hackathons, broadcasts and courses.

# Strategy
Shortly: first we check the main pages, then event pages

- Load `pages.json` - information about start wepbages: `name`, `url`, `func`
- Load `events.json` - information about current events:
    - `title`,
    - `desciption url`,
    - `registration url`,
    - `date`
    - `time`
    - `location`
    - `online status`
    - `cost`
    - `[tags]`
    - `language`
    - `speakers and themes (optional)`
    - `type (optional)`
- Check that events on the main pages are displayed in `events.json`
- If the event is not in the file, we need to add basic information about it on the second stage
- Past events are transfered to `past_events.json`
- On the next stage if any information is missing, we need to supplement it by going to specific url and parse the page

In [122]:
import json
import requests
from datetime import datetime, date
from bs4 import BeautifulSoup

In [125]:
date.today()

datetime.date(2020, 6, 1)

In [139]:
headers = requests.utils.default_headers()
current_date = date.today()

with open('pages.json') as pages_file:
    pages = json.load(pages_file)
    
with open('events.json') as events_file:
    events = json.load(events_file)

date_alias = {'rus':
                 {'weekdays':
                    {'short': ('пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс')},
                  'months':
                    {'gentive': ('января', 'февраля', 'марта',
                                 'апреля', 'мая', 'июня',
                                 'июля', 'августа', 'сентября',
                                 'октября', 'ноября', 'декабря')}
                 }
              }

    
class Event:
    def __init__(self, data):
        self.data = data
        self.update()
    
    def update(self):
        url = self.data['url']
        events[url] = {}
        for key in self.data:
            events[url][key] = self.data[key]
    

def main_parser():
    '''Evaluates other parser functions for websites from events.json'''
    for page in pages:
        name = pages[page]['name']
        url = pages[page]['url']
        func = pages[page]['func']
        eval(f'{func}("{name}", "{url}")')

In [140]:
def yandex_date_converter(date_string):
    '''Parsing date in format вт, 9 июня'''
    lang = 'rus'
    weekdays = date_alias[lang]['weekdays']['short']
    months = date_alias[lang]['months']['gentive']
    
    # преобразуем дату из строки
    weekday, event_day, month = date_string.split()
    event_day = int(event_day)
    event_month = months.index(month) + 1
    event_weekday = weekdays.index(weekday[:-1])
    event_year = current_date.year
    
    # рассматриваем случай конца года: слишком старые записи не хранят
    # но во второй половине года могут быть записи на следующий год
    if (event_month-current_date.month) < -3:
        event_year += 1

    event_date = date(event_year, event_month, event_day)
    if event_date.weekday() == event_weekday:
        return event_date.isoformat()
    

def yandex(name='Мероприятия Яндекса', url='https://events.yandex.ru/'):
    print(f'Parsing page {name} at {url}')
    page = requests.get(url, headers=headers).text
    soup = BeautifulSoup(page, 'html.parser')
    event_cards = soup.body.find_all('div', {'class':'event-card'})
    for event_card in event_cards:
        date_string = event_card.find('div', {'class':'event-card__date'}).text
        event_date = yandex_date_converter(date_string)
        event_url = event_card.a['href']
        if 'http' not in event_url:
            event_url = url[:-1] + event_url
        data = {'url':event_url,
                'title':event_card.a['title'],
                'date': event_date}
        event = Event(data)
        print(event.data)
    return event_cards


events_list = yandex()

Parsing page Мероприятия Яндекса at https://events.yandex.ru/
{'url': 'https://events.yandex.ru/events/vstrecha-rossijskoj-r-09-06-2020', 'title': 'Встреча Российской рабочей группы по стандартизации С++', 'date': '2020-06-09'}
{'url': 'https://events.yandex.ru/events/tutorial-catboost-04-06-2020', 'title': 'Туториал: новые фичи CatBoost', 'date': '2020-06-04'}
{'url': 'https://events.yandex.ru/events/toloka-04-06-2020', 'title': 'Вебинар по разметке больших массивов данных от Яндекс.Толоки и Dbrain', 'date': '2020-06-04'}
{'url': 'https://cloud.yandex.ru/events/137', 'title': 'Облако в сфере образования', 'date': '2020-06-04'}
{'url': 'http://yandex.ru/promo/events/online/kazakhstan?utm_source=events&utm_medium=post&utm_campaign=event', 'title': 'Яндекс для Казахстана: как продолжить бизнес в новой реальности', 'date': '2020-06-04'}
{'url': 'https://events.yandex.ru/events/hardware/29-may-2020', 'title': 'Я.Железо: разбираем Яндекс.Станцию Мини в прямом эфире', 'date': '2020-05-29'}
{

In [141]:
with open('events.json', 'w') as events_file:
    json.dump(events, events_file)